In [9]:
import urllib.request, json 
import pandas as pd
from pandas.io.json import json_normalize
import requests

resp = requests.get('https://www.metro.net/wp-json/wp/v2/line-override/', timeout=10,headers={'Content-Type': 'application/json'})
df= pd.read_json(resp.text)

main_df = json_normalize(df['acf'])
main_df
# cleaned_df = df.drop(['_links','yoast_head','yoast_head_json'],axis=1)

main_df.to_csv('custom-schedule-page.csv',index=False)


C:\Users\hikou\AppData\Local\Temp\ipykernel_8616\3940014327.py:9: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  main_df = json_normalize(df['acf'])
